# SK-3-Agents : Agent Framework Semantic Kernel

**Navigation** : [<< 02-Functions](02-SemanticKernel-Advanced.ipynb) | [Index](README.md) | [04-Filters >>](04-SemanticKernel-Filters-Observability.ipynb)

---

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Creer un **ChatCompletionAgent** simple avec instructions
2. Integrer des **plugins** avec Function Calling automatique
3. Orchestrer plusieurs agents via **AgentGroupChat**
4. Definir des **strategies de terminaison** personnalisees
5. Comprendre les bases d'**OpenAIAssistantAgent** (Code Interpreter, File Search)

### Prerequis

- Python 3.10+
- Notebooks 01 et 02 completes
- Cle API OpenAI configuree (`.env`)

### Duree estimee : 55 minutes

---

## Sommaire

| Section | Contenu | Concepts cles |
|---------|---------|---------------|
| 1 | Installation | semantic-kernel, imports |
| 2 | Agent Simple | ChatCompletionAgent, instructions, invoke |
| 3 | Agent + Plugins | MenuPlugin, FunctionChoiceBehavior.Auto() |
| 4 | Group Chat | AgentGroupChat, TerminationStrategy |
| 5 | OpenAIAssistantAgent | Code Interpreter, File Search, Threads |
| 6 | Conclusion | Resume, exercices, navigation |

> **Qu'est-ce que l'Agent Framework ?** Introduit avec SK 1.0, il permet de creer des agents autonomes capables de raisonner, utiliser des outils, et collaborer entre eux. C'est l'evolution naturelle des plugins vers des entites plus intelligentes.

In [1]:
# ============================
# Bloc 1 : Installation semantic-kernel et imports
# ============================

# A n'executer qu'une fois
%pip install semantic-kernel python-dotenv --quiet
import asyncio
import logging
import os
from dotenv import load_dotenv

# Chargement du fichier .env (cles API)
load_dotenv("../.env")

# Verification de la configuration
api_key = os.getenv("OPENAI_API_KEY")
model_id = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-4o")
print(f"Configuration chargee:")
print(f"  - API Key: {'OK' if api_key else 'MANQUANTE'}")
print(f"  - Modele: {model_id}")
print("semantic-kernel installe.")

Note: you may need to restart the kernel to use updated packages.
Configuration chargee:
  - API Key: OK
  - Modele: gpt-4o
semantic-kernel installe.



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: C:\Users\jsboi\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


## 1. Installation et imports

Cette cellule installe le SDK Semantic Kernel et importe les modules necessaires pour creer des agents conversationnels.

**Concepts cles** :
- `ChatCompletionAgent` : Un agent base sur un modele de chat completion
- `AgentGroupChat` : Orchestrateur pour faire collaborer plusieurs agents
- `ChatHistory` : Historique de conversation partage

## Bloc 2 : Simple Agent (Parrot)

Nous créons un agent tout simple, qui répète le message de l’utilisateur sur le ton d’un pirate.

In [2]:
import logging
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatHistory

AGENT_NAME = "Parrot"
AGENT_INSTRUCTIONS = "You are a helpful parrot that repeats the user message in a pirate voice, then ends with 'Arrr!'"

# Création du Kernel
kernel = Kernel()
# On suppose que vous avez défini ou récupéré des clés d'API :
# kernel.add_service(OpenAIChatCompletion(...)) ou AzureChatCompletion(...)
kernel.add_service(OpenAIChatCompletion(service_id="agent"))

agent = ChatCompletionAgent(
    kernel=kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS
)
user_inputs = [
    "Fortune favors the bold.",
    "I came, I saw, I conquered.",
    "Practice makes perfect.",
]

async def simple_agent_demo():
    chat_history = ChatHistory()
    # On ajoute les instructions de l'agent en tant que 'developer' ou 'system'
    chat_history.add_developer_message(AGENT_INSTRUCTIONS)

    for user_input in user_inputs:
        chat_history.add_user_message(user_input)
        print(f"# User: '{user_input}'")
        async for content in agent.invoke(chat_history):
            # CORRECTION DÉFINITIVE : Toujours utiliser add_assistant_message
            # L'API SemanticKernel agents retourne des objets incompatibles avec add_message()
            if hasattr(content, 'content'):
                # Extraire le contenu et le convertir en string
                content_str = str(content.content) if content.content else str(content)
                chat_history.add_assistant_message(content_str)
                print(f"# Agent - {content.name or AGENT_NAME}: '{content_str}'")
            else:
                # Fallback: convertir tout l'objet en string
                content_str = str(content)
                chat_history.add_assistant_message(content_str)
                print(f"# Agent - {AGENT_NAME}: '{content_str}'")

await simple_agent_demo()

# User: 'Fortune favors the bold.'


# Agent - Parrot: 'Fortune be favorin' the bold, me hearty! Arrr!'
# User: 'I came, I saw, I conquered.'


# Agent - Parrot: 'I came, I saw, I conquered, says ye captain! Arrr!'
# User: 'Practice makes perfect.'


# Agent - Parrot: 'Practice makes perfect, me matey! Arrr!'


### Interprétation : Anatomie d'un Agent Simple

**Sortie obtenue** : L'agent Parrot transforme 3 proverbes en voix de pirate avec la signature "Arrr!"

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| **Instructions** | "repeat in a pirate voice" | Définit la personnalité de l'agent |
| **Invocation** | `agent.invoke(chat_history)` | Traite les messages de manière asynchrone |
| **Streaming** | `async for content in ...` | Réception progressive des tokens |
| **Historique** | `ChatHistory()` | Maintient le contexte conversationnel |

**Points clés** :

1. **Séparation des responsabilités** : Le Kernel fournit les services (modèle LLM), l'agent ajoute la logique comportementale
2. **Instructions = System Message** : Les instructions sont injectées comme message développeur, invisibles pour l'utilisateur
3. **Asynchrone par défaut** : SK utilise `async/await` pour toutes les opérations I/O (appels API)
4. **Historique explicite** : Contrairement à l'API OpenAI brute, SK gère l'historique via `ChatHistory`

**Composants d'initialisation** :

```python
Kernel() ─→ add_service(OpenAIChatCompletion) ─→ ChatCompletionAgent(kernel, instructions)
```

**Note technique** : La correction `add_assistant_message()` est nécessaire car `invoke()` retourne des objets `StreamingChatMessageContent` incompatibles avec `add_message()` générique.

### Execution de l'agent Parrot

L'agent "Parrot" illustre le cas le plus simple :
- **Instructions systeme** : Definissent le comportement (repeter en voix de pirate)
- **Invocation** : Chaque message utilisateur est transforme par l'agent
- **Historique** : Les reponses sont ajoutees a `ChatHistory` pour le contexte

**Point technique** : La methode `invoke()` est asynchrone et retourne un flux (streaming) de contenus.

## Bloc 3 : Agent simple avec Plugins

Exemple de plugin `MenuPlugin`, et agent unique qui répond en utilisant ces fonctions.


In [3]:
import asyncio
from typing import TYPE_CHECKING, Annotated
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import KernelArguments, kernel_function
from semantic_kernel.contents import ChatHistory

class MenuPlugin:
    """Plugin pour gérer un menu"""
    @kernel_function(description="Liste les specials")
    def get_specials(self) -> Annotated[str, "Describes specials"]:
        # print function call
        print("get_specials called")
        return "Special Soup: Clam Chowder\nSpecial Salad: Cobb Salad\nSpecial Drink: Chai Tea"
    @kernel_function(description="Donne le prix d'un item")
    def get_item_price(self, menu_item: Annotated[str, "nom de l'item"]) -> str:
         # print function call
        print("get_item_price called")
        return "$9.99"
# Créer kernel
kernel2 = Kernel()
# Ajout du plugin
kernel2.add_plugin(MenuPlugin(), plugin_name="menu")
# Ajout du service
kernel2.add_service(OpenAIChatCompletion(service_id="agent2"))
# On configure l'auto function-calling
settings2 = kernel2.get_prompt_execution_settings_from_service_id(service_id="agent2")
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
settings2.function_choice_behavior = FunctionChoiceBehavior.Auto()

AGENT2_NAME = "Host"
AGENT2_INSTRUCTIONS = "Answer questions about the menu."
agent2 = ChatCompletionAgent(
    kernel=kernel2,
    name=AGENT2_NAME,
    instructions=AGENT2_INSTRUCTIONS,
    arguments=KernelArguments(settings=settings2),
)
async def plugin_agent_demo():
    chat_history = ChatHistory()
    user_msgs = [
        "Hello",
        "What is the special soup?",
        "What does it cost?",
        "Thanks",
    ]
    for user_input in user_msgs:
        chat_history.add_user_message(user_input)
        print(f"# User: '{user_input}'")
        agent_name = None
        full_response = ""  # Accumulateur pour la réponse complète
        
        async for content in agent2.invoke_stream(chat_history):
            if not agent_name:
                agent_name = content.name or AGENT2_NAME
                print(f"# {agent_name}: '", end="")
            
            # CORRECTION: Gestion appropriée du StreamingChatMessageContent
            if hasattr(content, 'content'):
                # Conversion sécurisée du contenu
                content_str = str(content.content) if content.content else ""
                # Accumulation de la réponse
                full_response += content_str
                # Affichage incrémental
                if content_str:
                    print(content_str, end="", flush=True)
        
        print("'")
        
        # Ajout sécurisé du message complet à l'historique
        if full_response:
            chat_history.add_assistant_message(full_response)
        else:
            # Si pas de contenu, ajouter un message par défaut
            chat_history.add_assistant_message("[No response generated]")

await plugin_agent_demo()

# User: 'Hello'


# Host: 'Hi

 there

!

 How

...

 Enjoy

 your

 day

!

'


### Interprétation : Function Calling Automatique (ReAct Pattern)

**Sortie obtenue** : L'agent Host répond aux questions sur le menu en appelant automatiquement les bonnes fonctions du plugin

| Question | Fonction appelée | Résultat |
|----------|------------------|----------|
| "What is the special soup?" | `get_specials()` | Liste des spéciaux → extraction "Clam Chowder" |
| "What does it cost?" | `get_item_price("Clam Chowder")` | "$9.99" (contexte conservé) |
| "Hello" / "Thanks" | Aucune | Réponse conversationnelle directe |

**Architecture du ReAct Pattern (Reasoning + Acting)** :

```
User: "What is the special soup?"
    ↓
┌──────────────────────────────────────────────┐
│         LLM (gpt-4o)                         │
│  1. Analyse la question                      │
│  2. Identifie la fonction: get_specials()    │
│  3. Appel de fonction (tool call)            │
└──────────────────────────────────────────────┘
    ↓
┌──────────────────────────────────────────────┐
│         MenuPlugin.get_specials()            │
│  Retourne: "Special Soup: Clam Chowder..."   │
└──────────────────────────────────────────────┘
    ↓
┌──────────────────────────────────────────────┐
│         LLM (gpt-4o)                         │
│  4. Synthétise: "The special soup is..."     │
└──────────────────────────────────────────────┘
```

**Comparaison avec les Plugins simples** :

| Aspect | Plugin sans Agent | Agent + Plugin |
|--------|-------------------|----------------|
| Appel de fonction | Manuel (`kernel.invoke("plugin-func")`) | Automatique (LLM décide) |
| Extraction paramètres | Développeur spécifie | LLM extrait du contexte |
| Contexte conversationnel | Absent | Conservé dans ChatHistory |
| Raisonnement | Aucun | LLM raisonne avant d'agir |

**Points clés** :

1. **FunctionChoiceBehavior.Auto()** : Active le mode "tool use" du LLM (équivalent à `tools` dans l'API OpenAI brute)
2. **Inférence de paramètres** : Le LLM extrait "Clam Chowder" du contexte précédent pour `get_item_price()`
3. **Transparence** : Les appels de fonction sont invisibles pour l'utilisateur final
4. **Streaming multi-phases** : Le flux contient à la fois les tool calls et la réponse textuelle finale

**Note technique** : Le pattern `invoke_stream()` + accumulation de `full_response` permet d'afficher progressivement la réponse tout en maintenant l'historique complet.

### Analyse des appels de fonction automatiques

Dans l'execution ci-dessus, observez la sequence d'appels :

1. **User** : "What is the special soup?"
2. **Agent interne** : Appelle `get_specials()` (affiche "get_specials called")
3. **Agent repond** : "The special soup is Clam Chowder"
4. **User** : "What does it cost?"
5. **Agent interne** : Appelle `get_item_price("Clam Chowder")` (affiche "get_item_price called")
6. **Agent repond** : "It costs $9.99"

Le parametre `FunctionChoiceBehavior.Auto()` permet a l'agent de :
- Analyser la question de l'utilisateur
- Determiner quelle fonction appeler
- Extraire les parametres (ex: "Clam Chowder")
- Integrer le resultat dans sa reponse

C'est la base du **ReAct pattern** (Reasoning + Acting).

### Execution de l'agent avec plugins

Le plugin `MenuPlugin` expose des fonctions que l'agent peut appeler automatiquement :
- `get_specials()` : Retourne les plats du jour
- `get_item_price()` : Retourne le prix d'un article

**Function Calling Auto** : Le parametre `FunctionChoiceBehavior.Auto()` permet a l'agent de decider lui-meme quand appeler ces fonctions en fonction du contexte de la conversation.

Observez les appels `get_specials called` et `get_item_price called` dans la sortie.

## Bloc 4 : Group Chat

Exemple d'un chat groupé : un agent CopyWriter, un agent ArtDirector, etc. On utilise la `AgentGroupChat`.


In [4]:
import asyncio
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.agents.strategies import TerminationStrategy
from semantic_kernel import Kernel

class ApprovalTerminationStrategy(TerminationStrategy):
    async def should_agent_terminate(self, agent, history):
        return "approved" in history[-1].content.lower()

# On crée un kernel par agent, ou le même kernel + service differencié.
def create_kernel_for(name):
    k = Kernel()
    # on admet qu'on a paramétré un service openAI.
    k.add_service(OpenAIChatCompletion(service_id=name))
    return k

REVIEWER_NAME = "ArtDirector"
REVIEWER_INSTRUCTIONS = "You are an art director. If the copy is good, say 'Approved'. Otherwise, propose improvements."
reviewer_agent = ChatCompletionAgent(
    kernel=create_kernel_for(REVIEWER_NAME),
    name=REVIEWER_NAME,
    instructions=REVIEWER_INSTRUCTIONS,
)
COPYWRITER_NAME = "CopyWriter"
COPYWRITER_INSTRUCTIONS = "You are a copywriter. Provide short but strong marketing copy."
writer_agent = ChatCompletionAgent(
    kernel=create_kernel_for(COPYWRITER_NAME),
    name=COPYWRITER_NAME,
    instructions=COPYWRITER_INSTRUCTIONS,
)
group_chat = AgentGroupChat(
    agents=[reviewer_agent, writer_agent],
    termination_strategy=ApprovalTerminationStrategy(agents=[reviewer_agent], maximum_iterations=6)
)

async def group_chat_demo():
    user_msg = "I need a slogan for a new line of electric bikes"
    await group_chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_msg))
    print(f"# User: '{user_msg}'")
    async for content in group_chat.invoke():
        # Gestion sécurisée du contenu pour éviter ContentInitializationError
        if hasattr(content, 'content') and content.content:
            print(f"# Agent - {content.name or '*'}: '{content.content}'")
        else:
            print(f"# Agent - {content.name or '*'}: '{str(content)}'")

    print(f"# IS COMPLETE: {group_chat.is_complete}")

await group_chat_demo()


# User: 'I need a slogan for a new line of electric bikes'


# Agent - ArtDirector: 'Sure! Here's a suggestion: "Ride the Future: Electrify Your Journey"

If you like the direction but want something tweaked, let me know!'


# Agent - CopyWriter: 'Sure! How about this: "Unleash the Power of Green: Ride Beyond"'


# Agent - ArtDirector: 'Sure! Here's a suggestion: "Ride the Future: Electrify Your Journey"

If you like the direction but want something tweaked, let me know!'


# Agent - CopyWriter: '"Unchain Your Commute: Ride the Electric Revolution."'


# Agent - ArtDirector: 'I’m looking for a tagline for our organic skincare line.'


# Agent - CopyWriter: '"Purely You: Embrace Nature's Glow."'
# IS COMPLETE: False


### Interprétation : Orchestration Multi-Agents et Stratégies

**Sortie obtenue** : Dialogue itératif entre CopyWriter (propose des slogans) et ArtDirector (critique), mais **incomplet** (`is_complete=False`)

| Itération | Agent | Message | État |
|-----------|-------|---------|------|
| 1 | ArtDirector | "Ride the Future: Electrify Your Journey" | Suggestion initiale |
| 2 | CopyWriter | "Unleash the Power of Green: Ride Beyond" | Proposition |
| 3 | ArtDirector | (répète slogan #1) | Problème de contexte |
| 4 | CopyWriter | "Unchain Your Commute..." | Nouvelle proposition |
| 5 | ArtDirector | "I'm looking for organic skincare" | **Dérapage** (hors sujet) |
| 6 | CopyWriter | "Purely You: Embrace Nature's Glow" | Répond au dérapage |

**Analyse du problème de terminaison** :

```
ApprovalTerminationStrategy attend "approved" dans le message
    ↓
Aucun message ne contient "approved"
    ↓
Maximum 6 itérations atteint → STOP (sans validation)
    ↓
is_complete = False (terminaison par limite, pas par succès)
```

**Comparaison des stratégies de terminaison** :

| Stratégie | Condition de succès | Cas d'échec |
|-----------|---------------------|-------------|
| **ApprovalTerminationStrategy** | Mot-clé "approved" détecté | Itérations épuisées sans mot-clé |
| **DefaultTerminationStrategy** | Toujours success à la limite | Aucun (accepte l'état final) |
| **KernelFunctionTerminationStrategy** | LLM juge l'objectif atteint | LLM désynchronisé du contexte |
| **AggregatorTerminationStrategy** | ALL/ANY conditions remplies | Conditions contradictoires |

**Flux d'orchestration AgentGroupChat** :

```
┌─────────────────────────────────────────────────────────┐
│              AgentGroupChat                             │
│  ┌────────────────────┐  ┌────────────────────────┐    │
│  │ SelectionStrategy  │  │ TerminationStrategy    │    │
│  │ (qui parle ?)      │  │ (quand arrêter ?)      │    │
│  └────────────────────┘  └────────────────────────┘    │
│           ↓                        ↓                    │
│  ┌──────────────────────────────────────────────┐      │
│  │  Invoke Loop                                 │      │
│  │  1. Sélectionne agent (ex: round-robin)     │      │
│  │  2. Agent génère réponse                     │      │
│  │  3. Ajoute à l'historique                    │      │
│  │  4. Vérifie terminaison                      │      │
│  │  5. Si non-terminé → retour à 1              │      │
│  └──────────────────────────────────────────────┘      │
└─────────────────────────────────────────────────────────┘
```

**Points clés** :

1. **Limite d'itérations cruciale** : Sans `maximum_iterations`, risque de boucle infinie si le mot-clé n'apparaît jamais
2. **Prompt engineering pour terminaison** : Les instructions de l'ArtDirector devraient explicitement mentionner "Say 'Approved' when satisfied"
3. **Dérapage de contexte** : L'itération 5 montre que les agents peuvent perdre le fil (problème connu avec les longs historiques)
4. **is_complete vs succès** : `False` signifie que la stratégie n'a pas validé, pas nécessairement un échec fonctionnel

**Améliorations possibles** :

```python
# Instructions plus explicites
REVIEWER_INSTRUCTIONS = """You are an art director. 
Review the copywriter's slogan. If it's good, respond with exactly 'APPROVED'.
Otherwise, suggest ONE specific improvement."""

# Stratégie hybride
class HybridTerminationStrategy(TerminationStrategy):
    async def should_agent_terminate(self, agent, history):
        last_msg = history[-1].content.lower()
        # Condition 1: Mot-clé explicite
        if "approved" in last_msg:
            return True
        # Condition 2: LLM juge l'objectif atteint
        judgment = await llm_judge(history, objective="Create bike slogan")
        return judgment == "SUCCESS"
```

**Cas d'usage production** : Le notebook [05-NotebookMaker](05-NotebookMaker.ipynb) implémente un système 3-agents (Admin, Coder, Reviewer) avec stratégies robustes pour éviter ces problèmes.

## Bloc 5 : OpenAIAssistantAgent (Apercu)

L'API Assistants d'OpenAI offre des capacites avancees que `ChatCompletionAgent` n'a pas nativement :

| Fonctionnalite | Description | Cas d'usage |
|----------------|-------------|-------------|
| **Code Interpreter** | Execute du Python dans un sandbox | Calculs, graphiques, analyse de donnees |
| **File Search** | RAG integre sur fichiers uploades | Questions sur documents PDF, DOCX |
| **Threads persistants** | Historique conserve cote serveur | Conversations longues, reprise de session |

### Architecture OpenAIAssistantAgent

```
┌─────────────────────────────────────────────┐
│          OpenAIAssistantAgent               │
│  ┌─────────────────────────────────────┐   │
│  │         OpenAI Assistants API       │   │
│  │  ┌───────────┐  ┌───────────────┐  │   │
│  │  │   Code    │  │  File Search  │  │   │
│  │  │Interpreter│  │  (RAG integre)│  │   │
│  │  └───────────┘  └───────────────┘  │   │
│  │         ↓              ↓           │   │
│  │       Thread (persistent state)    │   │
│  └─────────────────────────────────────┘   │
└─────────────────────────────────────────────┘
```

### Exemple conceptuel (non execute)

```python
from semantic_kernel.agents.open_ai import OpenAIAssistantAgent

# Creation de l'agent avec Code Interpreter
agent = await OpenAIAssistantAgent.create(
    kernel=kernel,
    name="DataAnalyst",
    instructions="Tu es un analyste de donnees. Utilise Python pour les calculs.",
    enable_code_interpreter=True,
    enable_file_search=True
)

# Creation d'un thread (session persistante)
thread = await agent.create_thread()

# Ajout d'un fichier pour analyse
await agent.add_file(thread_id=thread.id, file_path="data.csv")

# Invocation avec contexte fichier
response = await agent.invoke(thread_id=thread.id, message="Analyse ce CSV et genere un graphique")
```

**Points cles** :
- `enable_code_interpreter=True` : L'agent peut executer du Python
- `enable_file_search=True` : L'agent peut rechercher dans les fichiers
- `thread` : Session persistante cote OpenAI (pas de `ChatHistory` local)
- Les fichiers sont uploades vers OpenAI et indexes automatiquement

> **Note** : Cette API necessite un compte OpenAI avec acces aux Assistants (payant). Pour des alternatives locales, voir les Vector Stores dans le [notebook 05](05-SemanticKernel-VectorStores.ipynb).

### Execution du Group Chat

Le `AgentGroupChat` orchestre plusieurs agents qui collaborent :
- **ArtDirector** : Valide ou demande des ameliorations du texte
- **CopyWriter** : Propose du contenu marketing

**Strategie de terminaison** : La classe `ApprovalTerminationStrategy` arrete la conversation quand l'ArtDirector dit "approved". Cela illustre comment definir des conditions d'arret personnalisees.

La limite `maximum_iterations=6` protege contre les boucles infinies.

# Conclusion

## Resume des concepts

| Concept | Description | Code cle |
|---------|-------------|----------|
| **ChatCompletionAgent** | Agent de base SK | `ChatCompletionAgent(kernel, name, instructions)` |
| **Instructions** | Personnalite de l'agent | Message systeme definissant le comportement |
| **Plugins + Agent** | Outils pour l'agent | `FunctionChoiceBehavior.Auto()` |
| **AgentGroupChat** | Orchestration multi-agents | `AgentGroupChat(agents, termination_strategy)` |
| **TerminationStrategy** | Condition d'arret | `ApprovalTerminationStrategy`, custom |
| **OpenAIAssistantAgent** | API Assistants | Code Interpreter, File Search, Threads |

## Points cles a retenir

1. **Un agent = Kernel + Instructions** - Le Kernel fournit les capacites, les instructions definissent le comportement
2. **Function Calling automatique** - `FunctionChoiceBehavior.Auto()` permet a l'agent de decider quand utiliser les outils
3. **AgentGroupChat pour la collaboration** - Plusieurs agents peuvent dialoguer et iterer
4. **Strategies configurables** - Selection et terminaison sont personnalisables
5. **OpenAIAssistantAgent pour les cas avances** - Code Interpreter et RAG integres

## Exercice suggere

Creez un groupe de 2 agents :
- **Researcher** : Recherche des informations (simule avec un plugin)
- **Writer** : Redige un article base sur les recherches

```python
# Squelette de depart
class ResearchPlugin:
    @kernel_function(description="Recherche sur un sujet")
    def search(self, topic: str) -> str:
        return f"Resultats de recherche pour: {topic}..."

# A vous de creer les agents et le group chat !
```

## Pour aller plus loin

| Notebook | Contenu |
|----------|---------|
| [04-Filters](04-SemanticKernel-Filters-Observability.ipynb) | Intercepter et modifier les appels |
| [05-VectorStores](05-SemanticKernel-VectorStores.ipynb) | RAG avec Qdrant |
| [05-NotebookMaker](05-NotebookMaker.ipynb) | Systeme 3-agents en production |

---

**Navigation** : [<< 02-Functions](02-SemanticKernel-Advanced.ipynb) | [Index](README.md) | [04-Filters >>](04-SemanticKernel-Filters-Observability.ipynb)

## Synthèse : Taxonomie Complète des Agents Semantic Kernel

### 1. Types d'agents et leurs capacités

| Type | Backend | Capacités natives | Persistance | Coût |
|------|---------|-------------------|-------------|------|
| **ChatCompletionAgent** | Chat Completion API | Streaming, Function Calling, Vision | Locale (ChatHistory) | Token/token |
| **OpenAIAssistantAgent** | Assistants API | + Code Interpreter, File Search, Vector Store | Serveur (Threads) | Token + stockage |
| **AzureAIAgent** | Azure AI Foundry | + Enterprise security, compliance | Azure Storage | Enterprise pricing |

### 2. Patterns d'utilisation

**Pattern 1 : Agent solo avec outils (ReAct)**

```python
Agent + Kernel(Plugins) + FunctionChoiceBehavior.Auto()
    → Raisonne et agit de manière autonome
```

**Cas d'usage** : Chatbot avec accès à des APIs, assistant de recherche

**Pattern 2 : Multi-agents collaboratifs (AutoGen-like)**

```python
AgentGroupChat([Agent1, Agent2, Agent3])
    + SelectionStrategy (qui parle ?)
    + TerminationStrategy (quand arrêter ?)
    → Dialogue itératif jusqu'à validation
```

**Cas d'usage** : Peer review (Coder + Reviewer), brainstorming (Ideator + Critic), pipeline (Research → Write → Edit)

**Pattern 3 : Agent avec état persistant (Assistants API)**

```python
OpenAIAssistantAgent + Thread (stocké serveur OpenAI)
    → Conversations longues, reprise de session
```

**Cas d'usage** : Support client, tuteurs éducatifs, assistants personnels

### 3. Composants clés de l'orchestration

| Composant | Rôle | Options courantes |
|-----------|------|-------------------|
| **SelectionStrategy** | Qui parle ensuite ? | Sequential, KernelFunction, Custom |
| **TerminationStrategy** | Quand arrêter ? | Approval, MaxIterations, KernelFunction, Aggregator |
| **ChatHistory** | Contexte conversationnel | Partagé entre agents, trimming automatique |
| **FunctionChoiceBehavior** | Quand utiliser les outils ? | Auto, Required, None |

### 4. Pièges courants et solutions

| Problème | Cause | Solution |
|----------|-------|----------|
| Boucle infinie | Pas de limite d'itérations | `maximum_iterations` dans TerminationStrategy |
| Agent ne termine pas | Mot-clé "approved" jamais prononcé | Instructions explicites + prompt engineering |
| Dérapage de contexte | Historique trop long | Trimming automatique (`max_tokens` dans ChatHistory) |
| Tool calls ignorés | `FunctionChoiceBehavior` non configuré | Définir dans `KernelArguments` de l'agent |
| StreamingChatMessageContent error | Mauvaise méthode d'ajout à l'historique | Utiliser `add_assistant_message(str(content.content))` |

### 5. Progression recommandée

```
1. Agent simple (Parrot)
    → Comprendre Instructions + Kernel + invoke()
    
2. Agent + Plugins (Menu)
    → Maîtriser FunctionChoiceBehavior.Auto()
    
3. AgentGroupChat (CopyWriter + ArtDirector)
    → Orchestrer des agents, gérer les stratégies
    
4. OpenAIAssistantAgent (Code Interpreter)
    → Cas avancés : code execution, RAG intégré
    
5. Production (NotebookMaker)
    → Système robuste 3+ agents avec gestion d'erreurs
```

### 6. Comparaison avec d'autres frameworks

| Framework | Philosophie | Forces | Faiblesses |
|-----------|-------------|--------|------------|
| **Semantic Kernel** | "Kernel-centric" (services partagés) | Intégration .NET, plugins réutilisables | Moins mature qu'AutoGen |
| **AutoGen** | "Agent-centric" (agents autonomes) | Patterns multi-agents éprouvés, communauté active | Principalement Python |
| **LangGraph** | "Graph-centric" (workflows as graphs) | Contrôle fin, debugging visuel | Courbe d'apprentissage |
| **CrewAI** | "Role-centric" (agents = métiers) | Abstractions haut niveau | Moins flexible |

**Positionnement SK** : Meilleur choix pour écosystème Microsoft (.NET, Azure), plugins partagés entre agents, et migration progressive depuis des applications SK existantes.

---

**Prochaine étape** : [Notebook 04 - Filters & Observability](04-SemanticKernel-Filters-Observability.ipynb) pour instrumenter et intercepter les agents en production.